# Import 

In [3]:
# Import torch
import torch
import torch.nn as nn
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt

In [5]:
# TODO: Setup device agnostic code
from argparse import ArgumentParser

parser = ArgumentParser(description="computer vision model argument")
parser.add_argument("--disable-cuda",  type=bool, default=False, help="Choose cuda device to train model?")
parser.add_argument("--learning-rate", "-lr", type=float, default=.01, help="Learning rate")
parser.add_argument("--epochs", "-e", type=int, default=40, help="Epochs")
parser.add_argument("--MODEL-PATH", type=str, default="../../Module/models", help="Model save path")
parser.add_argument('--file', '-f', type=str)
args = parser.parse_args()

args.device = None
if not args.disable_cuda and torch.cuda.is_available():
    args.device = "cuda"
else:
    args.device = 'cpu'

In [9]:
import zipfile
import requests
from pathlib import Path

folder_data = Path("./data", parents=True, exist_ok=True)
image_name = "pizza_steak_sushi"
image_path = folder_data / image_name

url = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"


if image_path.is_dir():
    print(f"{image_name} Dataset has already been installed.")
else:
    print(f"{image_name} hasnt been install yet. Lets download...")
    image_path.mkdir(parents=True, exist_ok=True)
    
    with open(image_path / "pizza_steak_sushi.zip", "wb") as f:
        response = requests.get(url)
        print(f"{image_name}.zip is being downloaded...")
        f.write(response.content)
        
    with zipfile.ZipFile(image_path / "pizza_steak_sushi.zip") as zip_file:
        print("Unzipping pizza, steak, sushi data...")
        zip_file.extractall(image_path)

    print("Download Success")

pizza_steak_sushi hasnt been install yet. Lets download...
pizza_steak_sushi.zip is being downloaded...
Unzipping pizza, steak, sushi data...
Download Success


In [ ]:
import os

def walk_through_path(folder_path):
    for root, file, dirname in os.walk(folder_path):
        print(f"There are {len(dir)}")
    